In [ ]:
# Import the functions used in this project
import awesome_functions as af
import decode_utils as du
from sklearn.externals import joblib

# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
submission = pd.read_csv("../asset/sample_submission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("submission  data : ", submission.shape)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle as pkl
slack_url = pkl.load(open("Slack_url/send_url.pickle", "rb"))

import xgboost
from sklearn.metrics import log_loss
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix
def fitNaiveBayesModel(X, y):
    return MultinomialNB(alpha=0.0).fit(X, y)

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_test = test.copy()
df_submission = submission.copy()

df_train_dd = pd.read_csv("Feature_matrix/df_train_dd_201807291831.csv")
df_test_dd = pd.read_csv("Feature_matrix/df_test_dd_201807291855.csv")
df_train_fl = pd.read_csv("Feature_matrix/df_train_fl_201807291845.csv")
df_test_fl = pd.read_csv("Feature_matrix/df_test_fl_201807291908.csv")
# df_ratio = pd.read_csv("Feature_matrix/df_new_has_all_the_ratio_201808031807.csv")

Train data :  (647054, 7)
Test  data :  (653646, 6)
submission  data :  (95674, 39)


In [ ]:
vn_and_weekdays_cols = ["VisitNumber", "TripType", 'Return', 'Monday', 'Tuesday',\
                        'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

def get_pivor_df(df, col):
    """
        example) 
            df_train_cp = get_pivor_df(df_decoded, "Company") 이후에,
            df_train_cp = get_rid_of_zero_units(df_train_cp) 를 사용해서 음수를 없애줄것.
    """
    
    return pd.pivot_table(data= df, index="VisitNumber", fill_value=0,\
                          values="ScanCount", columns=col, aggfunc=np.sum)

def get_rid_of_zero_units(df):
    return pd.DataFrame(np.where(df < 0, 0, df), columns = df.columns)

In [ ]:
def getSpecifiedVisitNumberData(df_train, vn):
    display(df_train[df_train.VisitNumber == vn])
#     display(df_train_dd[df_train_dd.VisitNumber == vn])

In [ ]:
def get_real_category_cnt(df_, col = "DepartmentDescription", threshold=1.0):
    df = df_.copy()
    df_count = df.groupby(["VisitNumber", col]).sum()["ScanCount"].reset_index(name="Sc_sum")
    
    # sc_count가 음수인 아이템은 같은 날 산 아이템이라 볼 수 없다.
    # 리턴하기위해 가져온 것들이므로 월마트를 나갈 때 들고가는 아이템이라 보지않아도된다. 
    # 그런 아이템은 모두 nan으로 바꾸고 drop한다.
    df_count["Sc_sum"] = np.where(df_count["Sc_sum"] <= 0, np.nan, df_count["Sc_sum"])
    df_count = df_count.dropna()
    
    # 남은 아이템들만 더해주면 위에서 지정해준 컬럼의 유니크한 개수만 남는다.
    df_count = df_count.groupby("VisitNumber").sum()["Sc_sum"].reset_index(name="Sc_sum")
    
    return list(df_count[df_count.Sc_sum == threshold].VisitNumber.unique())

In [ ]:
def get_flat_type_user(df_, col = "DepartmentDescription"):
    """
        예:) vn_dd_more_than_one, vn_dd_one = get_flat_type_user(df_decoded) 반드시 df는 decoding 함수로 decoded된 것을 넣을 것.
    """
    df = df_.copy()
    df_count = df.groupby(["VisitNumber", col]).sum()["ScanCount"].reset_index(name="Sc_sum")
    df_count["Sc_sum"] = np.where(df_count["Sc_sum"] <= 0, np.nan, df_count["Sc_sum"])
    df_count = df_count.dropna()
    df_count_total = df_count.groupby("VisitNumber").sum()["Sc_sum"].reset_index(name="Total")
    df_merged = pd.merge(df_count, df_count_total, on="VisitNumber")
    df_merged["P"] = df_merged["Sc_sum"].div(df_merged["Total"])
#     df_merged = df_merged[df_merged["Total"] != 1.0]
    
    one_item_only_vn = []
    more_than_one_vn_count = []
    for i, vn in enumerate(df_merged.VisitNumber.unique()):
        tmp_df = df_merged[df_merged["VisitNumber"] == vn]
        unique_p = list(tmp_df["P"].unique())
        length_unique_p = len(unique_p)
        if length_unique_p == 1:
            if unique_p[0] == 1:
                one_item_only_vn.append(vn)
            else:
                more_than_one_vn_count.append(vn)
        if i % 5000 == 0:
            print(str(i) + "까지 진행됨.")
    
    return more_than_one_vn_count, one_item_only_vn

In [ ]:
# import decode_utils as du
# import awesome_functions as af
from sklearn.metrics import confusion_matrix
from IPython.display import display, Markdown

def getAccuracy(y_true, y_pred, data_length, i):
    """
        y_true : 원래 타겟 컬럼의 데이터를 넣어준다.
        y_pred : 예측한 값을 넣어준다.
        data_length : 예측한 데이터의 총 개수를 넣어준다.
    """
    accuracy = round(np.trace(confusion_matrix(y_true, y_pred))/data_length, 4)
    display(Markdown("##### Accuracy : " + str(accuracy) + ", smoothing : " + str(i)))
    return accuracy

def fitNaiveBayesModel_smoothing(X, y, a = 1.0):
    return MultinomialNB(alpha=a).fit(X, y)

def __combine_dd_fl(df_dd, df_fl):
    col_we_dont_need = ["VisitNumber", "TripType", 'Return', 'Monday',\
                        'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    cols_we_need = [col for col in df_dd.columns if col not in col_we_dont_need]
    cols = [col for col in cols_we_need if col != "VisitNumber"]
    return du.concatDf(df_dd[cols], df_fl)
    
def get_concat_df_for_test_df(df_dd, df_fl):
    return __combine_dd_fl(df_dd, df_fl)
    
def get_df_has_specified_vn_list(vn_list, df=None, df_dd=None, df_fl=None, is_in=True,
                                 is_need_to_make_df = False):
    if is_need_to_make_df:
        df = __combine_dd_fl(df_dd, df_fl)
        print("Concat 완료.")
    if is_in:
        return df[df.VisitNumber.isin(vn_list)]
    return df[~df.VisitNumber.isin(vn_list)]

def test_multinomial(df, early_stop = 5):
    X, y = af.get_df_to_fit(df)
    a = np.arange(0, 0.16, 0.01)
    accuracy_ = 0
    count = 0
    result = []
    alpha = []
    for i in a:
        if count >= early_stop:
            best_score = np.array(result).max()
            idx = result.index(best_score)
            display(Markdown("## Best_score : " + str(best_score) + ", Alpha : " + str(alpha[idx])))
            return
        model_nb = fitNaiveBayesModel_smoothing(X, y, i)
        y_pred = model_nb.predict(X)
        accuracy = getAccuracy(y, y_pred, len(X), i)
        if accuracy_ > accuracy:
            count += 1
        accuracy_ = accuracy
        result.append(accuracy)
        alpha.append(i)
    best_score = np.array(result).max()
    idx = result.index(best_score)
    display(Markdown("## Best_score : " + str(best_score) + ", Alpha : " + str(alpha[idx])))
#     return best_score, alpha[idx]

def cal_average_of_accuracy(li1, li2, score_li):    
    li3 = 96574 - len(li1 + li2)
    weight = [len(li1), len(li2), li3]
    result = 0
    for i, score in enumerate(score_li):
        result += weight[i] * score
    result = result/96574
    print(result)
    return result

In [ ]:
def saveModelObjectAsPickle(model, feature):
    filename = "feature_"+ str(feature) + "_xgb_trained_model.pkl"
    joblib.dump(model, filename)

In [ ]:
df_decoded = du.decodeStuffNeedsToBeDecoded(df_train)

In [ ]:
df_test_decoded = du.decodeStuffNeedsToBeDecoded(df_test)

In [10]:
# vn_dd_more_than_one, vn_dd_one = get_flat_type_user(df_decoded)

In [ ]:
vn_fl_more_than_one, vn_fl_one = get_flat_type_user(df_decoded, "FinelineNumber")

In [ ]:
vn_test_fl_more_than_one, vn_test_fl_one = get_flat_type_user(df_test_decoded, "FinelineNumber")

In [13]:
# vn_company_more_than_one, vn_company_one = get_flat_type_user(df_decoded, "Company")

In [17]:
cols = [col for col in df_train_dd.columns if col != "TripType"]

In [21]:
df_test_dd = df_test_dd[cols]

In [25]:
cols_fl = [col for col in df_train_fl.columns if col != "TripType"]

In [28]:
df_test_fl = df_test_fl[cols_fl]

# 효원님이 만든 데이터프레임으로 골라낸 feature 1

In [286]:
df_to_test = get_df_has_specified_vn_list(vn_dd_one_hw, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [287]:
test_multinomial(df_to_test)

##### Accuracy : 0.8809, smoothing : 0.0

##### Accuracy : 0.8846, smoothing : 0.01

##### Accuracy : 0.886, smoothing : 0.02

##### Accuracy : 0.8863, smoothing : 0.03

##### Accuracy : 0.8859, smoothing : 0.04

##### Accuracy : 0.8852, smoothing : 0.05

##### Accuracy : 0.8846, smoothing : 0.06

##### Accuracy : 0.8837, smoothing : 0.07

##### Accuracy : 0.8832, smoothing : 0.08

## Best_score : 0.8863, Alpha : 0.03

In [37]:
df_to_test = get_df_has_specified_vn_list(hw_li, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [38]:
test_multinomial(df_to_test)

##### Accuracy : 0.762, smoothing : 0.0

##### Accuracy : 0.7542, smoothing : 0.01

##### Accuracy : 0.758, smoothing : 0.02

##### Accuracy : 0.7608, smoothing : 0.03

##### Accuracy : 0.7641, smoothing : 0.04

...

##### Accuracy : 0.787, smoothing : 0.12

##### Accuracy : 0.7881, smoothing : 0.13

##### Accuracy : 0.7906, smoothing : 0.14

##### Accuracy : 0.7924, smoothing : 0.15

## Best_score : 0.7924, Alpha : 0.15

In [41]:
alpha = np.arange(0.15, 1, 0.05)
X, y = af.get_df_to_fit(df_to_test)

In [42]:
for i in alpha:
    model_nb_dd_one_01 = fitNaiveBayesModel_smoothing(X, y, i)
    y_pred_dd_one = model_nb_dd_one_01.predict(X)
    af.getAccuracy(y, y_pred_dd_one, len(X))
    print(i)

##### Accuracy : 0.7924

0.15


##### Accuracy : 0.7986

0.2


##### Accuracy : 0.8045

...

0.8500000000000002


##### Accuracy : 0.8298

0.9000000000000002


##### Accuracy : 0.8302

0.9500000000000003


# feature에 company 까지 더해서 11000개 정도의 컬럼으로 돌린결과

In [360]:
df_to_test = get_df_has_specified_vn_list(vn_dd_one, df=df_train_dd, is_need_to_make_df=False)

In [361]:
test_multinomial(df_to_test)

##### Accuracy : 0.7224, smoothing : 0.0

##### Accuracy : 0.7182, smoothing : 0.01

##### Accuracy : 0.7231, smoothing : 0.02

##### Accuracy : 0.7265, smoothing : 0.03

##### Accuracy : 0.7306, smoothing : 0.04

...

##### Accuracy : 0.7676, smoothing : 0.12

##### Accuracy : 0.7692, smoothing : 0.13

##### Accuracy : 0.7716, smoothing : 0.14

##### Accuracy : 0.7743, smoothing : 0.15

## Best_score : 0.7743, Alpha : 0.15

In [362]:
alpha = np.arange(1, 2, 0.05)

In [363]:
X, y = af.get_df_to_fit(df_to_test)

In [364]:
for i in alpha:
    model_nb_dd_one_01 = fitNaiveBayesModel_smoothing(X, y, i)
    y_pred_dd_one = model_nb_dd_one_01.predict(X)
    af.getAccuracy(y, y_pred_dd_one, len(X))
    print(i)

##### Accuracy : 0.8284

1.0


##### Accuracy : 0.8283

1.05


##### Accuracy : 0.8287

...

1.8500000000000008


##### Accuracy : 0.8163

1.9000000000000008


##### Accuracy : 0.8154

1.9500000000000008


In [354]:
test_multinomial(df_to_test)

##### Accuracy : 0.7734, smoothing : 0.0

##### Accuracy : 0.7517, smoothing : 0.01

##### Accuracy : 0.7496, smoothing : 0.02

##### Accuracy : 0.7525, smoothing : 0.03

##### Accuracy : 0.7547, smoothing : 0.04

...

##### Accuracy : 0.7842, smoothing : 0.12

##### Accuracy : 0.7888, smoothing : 0.13

##### Accuracy : 0.7922, smoothing : 0.14

##### Accuracy : 0.7957, smoothing : 0.15

## Best_score : 0.7957, Alpha : 0.15

# feature 1 에  카테고리 하나 산사람들을 다 포함시킴

In [296]:
df_to_test = get_df_has_specified_vn_list(vn_dd_one, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [297]:
test_multinomial(df_to_test)

##### Accuracy : 0.8018, smoothing : 0.0

##### Accuracy : 0.7946, smoothing : 0.01

##### Accuracy : 0.7978, smoothing : 0.02

##### Accuracy : 0.8001, smoothing : 0.03

##### Accuracy : 0.8041, smoothing : 0.04

...

##### Accuracy : 0.8286, smoothing : 0.12

##### Accuracy : 0.8305, smoothing : 0.13

##### Accuracy : 0.8327, smoothing : 0.14

##### Accuracy : 0.8347, smoothing : 0.15

## Best_score : 0.8347, Alpha : 0.15

In [302]:
alpha = np.arange(0.15, 1, 0.05)

In [307]:
alpha = np.arange(1, 2, 0.05)

In [304]:
X, y = af.get_df_to_fit(df_to_test)

In [308]:
for i in alpha:
    model_nb_dd_one_01 = fitNaiveBayesModel_smoothing(X, y, i)
    y_pred_dd_one = model_nb_dd_one_01.predict(X)
    af.getAccuracy(y, y_pred_dd_one, len(X))
    print(i)

##### Accuracy : 0.8696

1.0


##### Accuracy : 0.8695

1.05


##### Accuracy : 0.8702

...

1.8500000000000008


##### Accuracy : 0.8552

1.9000000000000008


##### Accuracy : 0.8539

1.9500000000000008


# feature 2 dd

In [309]:
df_to_test = get_df_has_specified_vn_list(vn_dd_more_than_one, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [310]:
test_multinomial(df_to_test)

##### Accuracy : 0.8229, smoothing : 0.0

##### Accuracy : 0.8163, smoothing : 0.01

##### Accuracy : 0.8143, smoothing : 0.02

##### Accuracy : 0.8136, smoothing : 0.03

##### Accuracy : 0.814, smoothing : 0.04

##### Accuracy : 0.8169, smoothing : 0.05

##### Accuracy : 0.8154, smoothing : 0.06

##### Accuracy : 0.8136, smoothing : 0.07

## Best_score : 0.8229, Alpha : 0.0

In [327]:
vn_dd_3 = vn_dd_one + vn_dd_more_than_one

In [328]:
df_to_test = get_df_has_specified_vn_list(vn_dd_3, is_in=False, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [329]:
test_multinomial(df_to_test)

##### Accuracy : 0.7021, smoothing : 0.0

##### Accuracy : 0.6605, smoothing : 0.01

##### Accuracy : 0.6535, smoothing : 0.02

##### Accuracy : 0.648, smoothing : 0.03

##### Accuracy : 0.6444, smoothing : 0.04

##### Accuracy : 0.6414, smoothing : 0.05

## Best_score : 0.7021, Alpha : 0.0

In [419]:
cal_average_of_accuracy(vn_dd_one, vn_dd_more_than_one, [0.8702, 0.8229, 0.7021])

0.7663370348126824


0.7663370348126824

# dd feature 1 dd만 이용

In [420]:
df_to_test = get_df_has_specified_vn_list(vn_dd_one, df=df_dd)

In [421]:
test_multinomial(df_to_test)

##### Accuracy : 0.7327, smoothing : 0.0

##### Accuracy : 0.7329, smoothing : 0.01

##### Accuracy : 0.7331, smoothing : 0.02

##### Accuracy : 0.7332, smoothing : 0.03

##### Accuracy : 0.7346, smoothing : 0.04

...

##### Accuracy : 0.7354, smoothing : 0.12

##### Accuracy : 0.7354, smoothing : 0.13

##### Accuracy : 0.7354, smoothing : 0.14

##### Accuracy : 0.7353, smoothing : 0.15

## Best_score : 0.7355, Alpha : 0.09

# dd feature 2 dd만 이용

In [422]:
df_to_test = get_df_has_specified_vn_list(vn_dd_more_than_one, df=df_dd)

In [423]:
test_multinomial(df_to_test)

##### Accuracy : 0.5806, smoothing : 0.0

##### Accuracy : 0.5804, smoothing : 0.01

##### Accuracy : 0.581, smoothing : 0.02

##### Accuracy : 0.5813, smoothing : 0.03

##### Accuracy : 0.5813, smoothing : 0.04

...

##### Accuracy : 0.5811, smoothing : 0.12

##### Accuracy : 0.5811, smoothing : 0.13

##### Accuracy : 0.5811, smoothing : 0.14

##### Accuracy : 0.5813, smoothing : 0.15

## Best_score : 0.5813, Alpha : 0.03

# dd feature 3 dd만 이용

In [424]:
df_to_test = get_df_has_specified_vn_list(vn_dd_3, df=df_dd)

In [425]:
test_multinomial(df_to_test)

##### Accuracy : 0.6796, smoothing : 0.0

##### Accuracy : 0.6794, smoothing : 0.01

##### Accuracy : 0.6795, smoothing : 0.02

##### Accuracy : 0.6795, smoothing : 0.03

##### Accuracy : 0.6795, smoothing : 0.04

...

##### Accuracy : 0.6793, smoothing : 0.12

##### Accuracy : 0.6792, smoothing : 0.13

##### Accuracy : 0.6795, smoothing : 0.14

##### Accuracy : 0.6795, smoothing : 0.15

## Best_score : 0.6796, Alpha : 0.0

In [426]:
X, y = af.get_df_to_fit(df_to_test)

In [438]:
alpha = np.arange(20, 25, 0.05)

In [439]:
for i in alpha:
    model_nb_dd_one_01 = fitNaiveBayesModel_smoothing(X, y, i)
    y_pred_dd_one = model_nb_dd_one_01.predict(X)
    af.getAccuracy(y, y_pred_dd_one, len(X))
    print(i)

##### Accuracy : 0.6975

20.0


##### Accuracy : 0.6976

20.05


##### Accuracy : 0.6976

...

24.85000000000007


##### Accuracy : 0.6972

24.90000000000007


##### Accuracy : 0.6972

24.95000000000007


In [440]:
cal_average_of_accuracy(vn_dd_one, vn_dd_more_than_one, [0.7335, 0.5813, 0.6984])

0.6928343322219231


0.6928343322219231

# dd feature 1 fl만 사용

In [441]:
df_to_test = get_df_has_specified_vn_list(vn_dd_one, df=df_train_fl)

In [442]:
test_multinomial(df_to_test)

##### Accuracy : 0.8307, smoothing : 0.0

##### Accuracy : 0.8334, smoothing : 0.01

##### Accuracy : 0.8349, smoothing : 0.02

##### Accuracy : 0.8366, smoothing : 0.03

##### Accuracy : 0.8377, smoothing : 0.04

...

##### Accuracy : 0.8428, smoothing : 0.12

##### Accuracy : 0.8427, smoothing : 0.13

##### Accuracy : 0.8425, smoothing : 0.14

##### Accuracy : 0.8426, smoothing : 0.15

## Best_score : 0.8428, Alpha : 0.12

# dd feature 2 fl만 사용

In [443]:
df_to_test = get_df_has_specified_vn_list(vn_dd_more_than_one, df=df_train_fl)

In [444]:
test_multinomial(df_to_test)

##### Accuracy : 0.8243, smoothing : 0.0

##### Accuracy : 0.8257, smoothing : 0.01

##### Accuracy : 0.8254, smoothing : 0.02

##### Accuracy : 0.822, smoothing : 0.03

##### Accuracy : 0.8186, smoothing : 0.04

##### Accuracy : 0.8136, smoothing : 0.05

##### Accuracy : 0.8092, smoothing : 0.06

## Best_score : 0.8257, Alpha : 0.01

# dd feature 3 fl만 사용

In [446]:
df_to_test = get_df_has_specified_vn_list(vn_dd_3, is_in = False, df=df_train_fl)

In [447]:
test_multinomial(df_to_test)

##### Accuracy : 0.7123, smoothing : 0.0

##### Accuracy : 0.6932, smoothing : 0.01

##### Accuracy : 0.6876, smoothing : 0.02

##### Accuracy : 0.6838, smoothing : 0.03

##### Accuracy : 0.6808, smoothing : 0.04

##### Accuracy : 0.6781, smoothing : 0.05

## Best_score : 0.7123, Alpha : 0.0

In [448]:
cal_average_of_accuracy(vn_dd_one, vn_dd_more_than_one, [0.8428, 0.8257, 0.7123])

0.7647850394516121


0.7647850394516121

# feature fl 1

In [316]:
df_to_test = get_df_has_specified_vn_list(vn_fl_one, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [317]:
test_multinomial(df_to_test)

##### Accuracy : 0.8923, smoothing : 0.0

##### Accuracy : 0.9014, smoothing : 0.01

##### Accuracy : 0.9043, smoothing : 0.02

##### Accuracy : 0.9069, smoothing : 0.03

##### Accuracy : 0.9095, smoothing : 0.04

...

##### Accuracy : 0.9206, smoothing : 0.12

##### Accuracy : 0.9208, smoothing : 0.13

##### Accuracy : 0.9212, smoothing : 0.14

##### Accuracy : 0.9212, smoothing : 0.15

## Best_score : 0.9212, Alpha : 0.14

In [318]:
alpha = np.arange(0.15, 1, 0.05)
X, y = af.get_df_to_fit(df_to_test)

In [319]:
for i in alpha:
    model_nb_dd_one_01 = fitNaiveBayesModel_smoothing(X, y, i)
    y_pred_dd_one = model_nb_dd_one_01.predict(X)
    af.getAccuracy(y, y_pred_dd_one, len(X))
    print(i)

##### Accuracy : 0.9212

0.15


##### Accuracy : 0.9205

0.2


##### Accuracy : 0.9171

...

0.8500000000000002


##### Accuracy : 0.8721

0.9000000000000002


##### Accuracy : 0.8705

0.9500000000000003


# feature fl 2

In [320]:
df_to_test = get_df_has_specified_vn_list(vn_fl_more_than_one, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [321]:
test_multinomial(df_to_test)

##### Accuracy : 0.7325, smoothing : 0.0

##### Accuracy : 0.7117, smoothing : 0.01

##### Accuracy : 0.7048, smoothing : 0.02

##### Accuracy : 0.7009, smoothing : 0.03

##### Accuracy : 0.6981, smoothing : 0.04

##### Accuracy : 0.6957, smoothing : 0.05

## Best_score : 0.7325, Alpha : 0.0

# feature fl 3

In [334]:
vn_fl_3 = vn_fl_one + vn_fl_more_than_one

In [335]:
df_to_test = get_df_has_specified_vn_list(vn_fl_3, is_in=False, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [336]:
test_multinomial(df_to_test)

##### Accuracy : 0.7369, smoothing : 0.0

##### Accuracy : 0.6908, smoothing : 0.01

##### Accuracy : 0.6833, smoothing : 0.02

##### Accuracy : 0.678, smoothing : 0.03

##### Accuracy : 0.6738, smoothing : 0.04

##### Accuracy : 0.6701, smoothing : 0.05

## Best_score : 0.7369, Alpha : 0.0

In [337]:
cal_average_of_accuracy(vn_fl_one, vn_fl_more_than_one, [0.9212, 0.7325, 0.7369])

0.7727481754923685


0.7727481754923685

# feature fl 오직 fl만 써서 모델 만들기.

In [389]:
df_to_test = get_df_has_specified_vn_list(vn_fl_one, df=df_train_fl)

In [390]:
test_multinomial(df_to_test)

##### Accuracy : 0.8388, smoothing : 0.0

##### Accuracy : 0.8417, smoothing : 0.01

##### Accuracy : 0.8442, smoothing : 0.02

##### Accuracy : 0.8452, smoothing : 0.03

##### Accuracy : 0.8456, smoothing : 0.04

...

##### Accuracy : 0.8451, smoothing : 0.08

##### Accuracy : 0.8439, smoothing : 0.09

##### Accuracy : 0.8425, smoothing : 0.1

##### Accuracy : 0.8414, smoothing : 0.11

## Best_score : 0.8467, Alpha : 0.06

# feature 2 fl 오직 fl만 써서 모델 만들기.

In [400]:
df_to_test = get_df_has_specified_vn_list(vn_fl_more_than_one, df=df_train_fl)

In [401]:
test_multinomial(df_to_test)

##### Accuracy : 0.7649, smoothing : 0.0

##### Accuracy : 0.7623, smoothing : 0.01

##### Accuracy : 0.7602, smoothing : 0.02

##### Accuracy : 0.7593, smoothing : 0.03

##### Accuracy : 0.7575, smoothing : 0.04

##### Accuracy : 0.7568, smoothing : 0.05

## Best_score : 0.7649, Alpha : 0.0

# feature 3 fl 오직 fl만 써서 모델 만들기.

In [411]:
df_to_test = get_df_has_specified_vn_list(vn_fl_3, is_in=False, df=df_train_fl)

In [412]:
test_multinomial(df_to_test)

##### Accuracy : 0.7488, smoothing : 0.0

##### Accuracy : 0.7247, smoothing : 0.01

##### Accuracy : 0.7186, smoothing : 0.02

##### Accuracy : 0.7142, smoothing : 0.03

##### Accuracy : 0.7108, smoothing : 0.04

##### Accuracy : 0.7071, smoothing : 0.05

## Best_score : 0.7488, Alpha : 0.0

In [414]:
cal_average_of_accuracy(vn_fl_one, vn_fl_more_than_one, [0.8467, 0.7649, 0.7488])

0.7738112380143723


0.7738112380143723

# fl기준으로 나눈 vn경우에 1은 dd, fl로 같이 돌리고, 2, 3은 fl만 두고 돌리면 최고 점수로 0.7889이다.

In [415]:
cal_average_of_accuracy(vn_fl_one, vn_fl_more_than_one, [0.9212, 0.7649, 0.7488])

0.7888780209994408


0.7888780209994408

In [28]:
df_to_test = get_df_has_specified_vn_list(vn_fl_one, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [34]:
test_multinomial(df_to_test)

##### Accuracy : 0.9331, smoothing : 0.0

##### Accuracy : 0.9372, smoothing : 0.01

##### Accuracy : 0.9387, smoothing : 0.02

##### Accuracy : 0.9383, smoothing : 0.03

##### Accuracy : 0.9385, smoothing : 0.04

##### Accuracy : 0.9374, smoothing : 0.05

##### Accuracy : 0.9365, smoothing : 0.06

##### Accuracy : 0.9355, smoothing : 0.07

##### Accuracy : 0.9351, smoothing : 0.08

## Best_score : 0.9387, Alpha : 0.02

In [35]:
df_to_test = get_df_has_specified_vn_list(vn_dd_one, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [36]:
test_multinomial(df_to_test)

##### Accuracy : 0.9336, smoothing : 0.0

##### Accuracy : 0.9377, smoothing : 0.01

##### Accuracy : 0.9395, smoothing : 0.02

##### Accuracy : 0.939, smoothing : 0.03

##### Accuracy : 0.9388, smoothing : 0.04

##### Accuracy : 0.9377, smoothing : 0.05

##### Accuracy : 0.9367, smoothing : 0.06

##### Accuracy : 0.9357, smoothing : 0.07

## Best_score : 0.9395, Alpha : 0.02

In [142]:
len(more_than_one_vn_count), len(vn_dd_one), len(more_than_one_vn_count) + len(vn_dd_one)

(24922, 15597, 40519)

In [197]:
for vn in vn_fl_one:
    if vn in more_than_one_fl_vn_count:
        print(vn)

In [143]:
for vn in vn_dd_one:
    if vn in more_than_one_vn_count:
        print(vn)

0

In [128]:
getSpecifiedVisitNumberData(df_count, 103089)

,VisitNumber,DepartmentDescription,Sc_sum
172684,103089,MENS WEAR,0.0
172685,103089,NULL,1.0


In [126]:
getSpecifiedVisitNumberData(df_decoded, 103089)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Company
347786,999,103089,Sunday,000828313269,-1,MENS WEAR,1544,000828
347787,999,103089,Sunday,000000-99990,1,NULL,1.1,000000
347788,999,103089,Sunday,000828313252,1,MENS WEAR,1544,000828


In [125]:
getSpecifiedVisitNumberData(df_decoded, 76155)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Company
256570,999,76155,Wednesday,036000318616,1,INFANT CONSUMABLE HARDLINES,9007,036000
256571,999,76155,Wednesday,036000318616,-1,INFANT CONSUMABLE HARDLINES,9007,036000
256572,999,76155,Wednesday,000000-99990,1,NULL,1.1,000000


# feature 2

In [144]:
df_to_test = get_df_has_specified_vn_list(more_than_one_vn_count, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [145]:
test_multinomial(df_to_test)

##### Accuracy : 0.7609, smoothing : 0.0

##### Accuracy : 0.7424, smoothing : 0.01

##### Accuracy : 0.7364, smoothing : 0.02

##### Accuracy : 0.7322, smoothing : 0.03

##### Accuracy : 0.73, smoothing : 0.04

##### Accuracy : 0.7276, smoothing : 0.05

## Best_score : 0.7609, Alpha : 0.0

# feature 3

In [217]:
df_to_test = get_df_has_specified_vn_list(new_dd_li_to_test, is_in = False,
                                          df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [218]:
test_multinomial(df_to_test)

##### Accuracy : 0.7022, smoothing : 0.0

##### Accuracy : 0.6608, smoothing : 0.01

##### Accuracy : 0.6537, smoothing : 0.02

##### Accuracy : 0.6485, smoothing : 0.03

##### Accuracy : 0.6449, smoothing : 0.04

##### Accuracy : 0.6419, smoothing : 0.05

## Best_score : 0.7022, Alpha : 0.0

In [183]:
df_to_test = get_df_has_specified_vn_list(more_than_one_fl_vn_count, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


# feature 2

In [177]:
test_multinomial(df_to_test)

##### Accuracy : 0.72, smoothing : 0.0

##### Accuracy : 0.7011, smoothing : 0.01

##### Accuracy : 0.6959, smoothing : 0.02

##### Accuracy : 0.6932, smoothing : 0.03

##### Accuracy : 0.6898, smoothing : 0.04

##### Accuracy : 0.6877, smoothing : 0.05

## Best_score : 0.72, Alpha : 0.0

In [209]:
df_to_test = get_df_has_specified_vn_list(new_li_to_test, is_in=False,
                                          df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


# feature 3

In [210]:
test_multinomial(df_to_test)

##### Accuracy : 0.7369, smoothing : 0.0

##### Accuracy : 0.6908, smoothing : 0.01

##### Accuracy : 0.6833, smoothing : 0.02

##### Accuracy : 0.678, smoothing : 0.03

##### Accuracy : 0.6738, smoothing : 0.04

##### Accuracy : 0.6701, smoothing : 0.05

## Best_score : 0.7369, Alpha : 0.0

# feature 1 + 2

In [202]:
df_to_test = get_df_has_specified_vn_list(new_li_to_test, is_in=True,
                                          df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [203]:
test_multinomial(df_to_test)

##### Accuracy : 0.7498, smoothing : 0.0

##### Accuracy : 0.7377, smoothing : 0.01

##### Accuracy : 0.7349, smoothing : 0.02

##### Accuracy : 0.7337, smoothing : 0.03

##### Accuracy : 0.7325, smoothing : 0.04

##### Accuracy : 0.7314, smoothing : 0.05

## Best_score : 0.7498, Alpha : 0.0

# feature 2 + 3

In [204]:
df_to_test = get_df_has_specified_vn_list(vn_fl_one, is_in=False,
                                          df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [205]:
test_multinomial(df_to_test)

##### Accuracy : 0.6692, smoothing : 0.0

##### Accuracy : 0.6429, smoothing : 0.01

##### Accuracy : 0.6385, smoothing : 0.02

##### Accuracy : 0.635, smoothing : 0.03

##### Accuracy : 0.6325, smoothing : 0.04

##### Accuracy : 0.6306, smoothing : 0.05

## Best_score : 0.6692, Alpha : 0.0

# Company feature 1

In [378]:
df_to_test = get_df_has_specified_vn_list(vn_company_one, df=df_train_dd, is_need_to_make_df=False)

In [379]:
test_multinomial(df_to_test)

##### Accuracy : 0.7974, smoothing : 0.0

##### Accuracy : 0.8111, smoothing : 0.01

##### Accuracy : 0.8213, smoothing : 0.02

##### Accuracy : 0.829, smoothing : 0.03

##### Accuracy : 0.8357, smoothing : 0.04

...

##### Accuracy : 0.8601, smoothing : 0.12

##### Accuracy : 0.8596, smoothing : 0.13

##### Accuracy : 0.8601, smoothing : 0.14

##### Accuracy : 0.8617, smoothing : 0.15

## Best_score : 0.8617, Alpha : 0.15

In [380]:
alpha = np.arange(0.15, 1, 0.05)

In [381]:
X, y = af.get_df_to_fit(df_to_test)
for i in alpha:
    model_nb_dd_one_01 = fitNaiveBayesModel_smoothing(X, y, i)
    y_pred_dd_one = model_nb_dd_one_01.predict(X)
    af.getAccuracy(y, y_pred_dd_one, len(X))
    print(i)

##### Accuracy : 0.8617

0.15


##### Accuracy : 0.8661

0.2


##### Accuracy : 0.8669

...

0.8500000000000002


##### Accuracy : 0.8377

0.9000000000000002


##### Accuracy : 0.8361

0.9500000000000003


# Company feature 2

In [382]:
df_to_test = get_df_has_specified_vn_list(vn_company_more_than_one, df=df_train_dd, is_need_to_make_df=False)

In [383]:
test_multinomial(df_to_test)

##### Accuracy : 0.6859, smoothing : 0.0

##### Accuracy : 0.6649, smoothing : 0.01

##### Accuracy : 0.6597, smoothing : 0.02

##### Accuracy : 0.6569, smoothing : 0.03

##### Accuracy : 0.6559, smoothing : 0.04

##### Accuracy : 0.6544, smoothing : 0.05

## Best_score : 0.6859, Alpha : 0.0

# Company feature 3

In [384]:
vn_company_3 = vn_company_one + vn_company_more_than_one

In [385]:
df_to_test = get_df_has_specified_vn_list(vn_company_3, is_in=False, df=df_train_dd, is_need_to_make_df=False)

In [386]:
test_multinomial(df_to_test)

##### Accuracy : 0.7006, smoothing : 0.0

##### Accuracy : 0.6518, smoothing : 0.01

##### Accuracy : 0.6441, smoothing : 0.02

##### Accuracy : 0.6394, smoothing : 0.03

##### Accuracy : 0.6361, smoothing : 0.04

##### Accuracy : 0.633, smoothing : 0.05

## Best_score : 0.7006, Alpha : 0.0

In [387]:
cal_average_of_accuracy(vn_company_one, vn_company_more_than_one, [0.8669, 0.6859, 0.7006])

0.7335877379004702


0.7335877379004702

# Company feature는 dd_dummies, fl_dummies

In [392]:
df_to_test = get_df_has_specified_vn_list(vn_company_one, df_dd=df_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [393]:
test_multinomial(df_to_test)

##### Accuracy : 0.8809, smoothing : 0.0

##### Accuracy : 0.8864, smoothing : 0.01

##### Accuracy : 0.8925, smoothing : 0.02

##### Accuracy : 0.8972, smoothing : 0.03

##### Accuracy : 0.8995, smoothing : 0.04

...

##### Accuracy : 0.9121, smoothing : 0.12

##### Accuracy : 0.9132, smoothing : 0.13

##### Accuracy : 0.9133, smoothing : 0.14

##### Accuracy : 0.9139, smoothing : 0.15

## Best_score : 0.9139, Alpha : 0.15

In [394]:
X, y = af.get_df_to_fit(df_to_test)
for i in alpha:
    model_nb_dd_one_01 = fitNaiveBayesModel_smoothing(X, y, i)
    y_pred_dd_one = model_nb_dd_one_01.predict(X)
    af.getAccuracy(y, y_pred_dd_one, len(X))
    print(i)

##### Accuracy : 0.9139

0.15


##### Accuracy : 0.9157

0.2


##### Accuracy : 0.915

...

0.8500000000000002


##### Accuracy : 0.8742

0.9000000000000002


##### Accuracy : 0.8726

0.9500000000000003


# Company feature 2 dd_dummies, fl_dummies

In [395]:
df_to_test = get_df_has_specified_vn_list(vn_company_more_than_one, df_dd=df_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [396]:
test_multinomial(df_to_test)

##### Accuracy : 0.7494, smoothing : 0.0

##### Accuracy : 0.731, smoothing : 0.01

##### Accuracy : 0.7258, smoothing : 0.02

##### Accuracy : 0.7215, smoothing : 0.03

##### Accuracy : 0.7187, smoothing : 0.04

##### Accuracy : 0.7172, smoothing : 0.05

## Best_score : 0.7494, Alpha : 0.0

# Company feature 3 dd_dummies, fl_dummies

In [397]:
df_to_test = get_df_has_specified_vn_list(vn_company_3, is_in=False, df_dd=df_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [398]:
test_multinomial(df_to_test)

##### Accuracy : 0.7242, smoothing : 0.0

##### Accuracy : 0.6809, smoothing : 0.01

##### Accuracy : 0.6737, smoothing : 0.02

##### Accuracy : 0.6682, smoothing : 0.03

##### Accuracy : 0.6648, smoothing : 0.04

##### Accuracy : 0.6621, smoothing : 0.05

## Best_score : 0.7242, Alpha : 0.0

In [399]:
cal_average_of_accuracy(vn_company_one, vn_company_more_than_one, [0.9157, 0.7494, 0.7242])

0.773187957421252


0.773187957421252

# Company feature 1 fl_dummies

In [407]:
df_to_test = get_df_has_specified_vn_list(vn_company_one, df=df_train_fl)

In [408]:
test_multinomial(df_to_test)

##### Accuracy : 0.8435, smoothing : 0.0

##### Accuracy : 0.8468, smoothing : 0.01

##### Accuracy : 0.8485, smoothing : 0.02

##### Accuracy : 0.8491, smoothing : 0.03

##### Accuracy : 0.8496, smoothing : 0.04

...

##### Accuracy : 0.8481, smoothing : 0.09

##### Accuracy : 0.8473, smoothing : 0.1

##### Accuracy : 0.8474, smoothing : 0.11

##### Accuracy : 0.8471, smoothing : 0.12

## Best_score : 0.8501, Alpha : 0.06

# Company feature 2 fl_dummies

In [409]:
df_to_test = get_df_has_specified_vn_list(vn_company_more_than_one, df=df_train_fl)

In [410]:
test_multinomial(df_to_test)

##### Accuracy : 0.7818, smoothing : 0.0

##### Accuracy : 0.7801, smoothing : 0.01

##### Accuracy : 0.7801, smoothing : 0.02

##### Accuracy : 0.7787, smoothing : 0.03

##### Accuracy : 0.7775, smoothing : 0.04

##### Accuracy : 0.7754, smoothing : 0.05

##### Accuracy : 0.7738, smoothing : 0.06

## Best_score : 0.7818, Alpha : 0.0

# Company feature 3 fl_dummies

In [416]:
df_to_test = get_df_has_specified_vn_list(vn_company_3, is_in=False, df=df_train_fl)

In [417]:
test_multinomial(df_to_test)

##### Accuracy : 0.7345, smoothing : 0.0

##### Accuracy : 0.7128, smoothing : 0.01

##### Accuracy : 0.7067, smoothing : 0.02

##### Accuracy : 0.7025, smoothing : 0.03

##### Accuracy : 0.6985, smoothing : 0.04

##### Accuracy : 0.6954, smoothing : 0.05

## Best_score : 0.7345, Alpha : 0.0

In [418]:
cal_average_of_accuracy(vn_company_one, vn_company_more_than_one, [0.8501, 0.7818, 0.7345])

0.7724514941909831


0.7724514941909831

In [13]:
cal_average_of_accuracy(vn_company_one, vn_company_more_than_one, [0.9139, 0.7818, 0.7345])

0.7865797978752045


0.7865797978752045

# Upc feature 1 dd+fl

In [611]:
df_to_test = get_df_has_specified_vn_list(vn_upc_one, df_dd=df_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [612]:
test_multinomial(df_to_test)

##### Accuracy : 0.8973, smoothing : 0.0

##### Accuracy : 0.9033, smoothing : 0.01

##### Accuracy : 0.9071, smoothing : 0.02

##### Accuracy : 0.909, smoothing : 0.03

##### Accuracy : 0.9126, smoothing : 0.04

...

##### Accuracy : 0.9189, smoothing : 0.12

##### Accuracy : 0.9186, smoothing : 0.13

##### Accuracy : 0.9179, smoothing : 0.14

##### Accuracy : 0.9172, smoothing : 0.15

## Best_score : 0.9189, Alpha : 0.12

In [614]:
alpha = np.arange(0.15, 1, 0.05)

In [615]:
X, y = af.get_df_to_fit(df_to_test)
for i in alpha:
    model_nb_dd_one_01 = fitNaiveBayesModel_smoothing(X, y, i)
    y_pred_dd_one = model_nb_dd_one_01.predict(X)
    af.getAccuracy(y, y_pred_dd_one, len(X))
    print(i)

##### Accuracy : 0.9172

0.15


##### Accuracy : 0.9134

0.2


##### Accuracy : 0.9102

...

0.8500000000000002


##### Accuracy : 0.862

0.9000000000000002


##### Accuracy : 0.8604

0.9500000000000003


# Upc feature 2 dd+fl

In [616]:
df_to_test = get_df_has_specified_vn_list(vn_upc_more_than_one, df_dd=df_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [617]:
test_multinomial(df_to_test)

##### Accuracy : 0.6985, smoothing : 0.0

##### Accuracy : 0.6728, smoothing : 0.01

##### Accuracy : 0.6661, smoothing : 0.02

##### Accuracy : 0.6622, smoothing : 0.03

##### Accuracy : 0.6598, smoothing : 0.04

##### Accuracy : 0.6574, smoothing : 0.05

## Best_score : 0.6985, Alpha : 0.0

# Upc feature 3 dd+fl

In [618]:
vn_upc_3 = vn_upc_one + vn_upc_more_than_one

In [619]:
df_to_test = get_df_has_specified_vn_list(vn_upc_3, is_in=False, df_dd=df_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [620]:
test_multinomial(df_to_test)

##### Accuracy : 0.7615, smoothing : 0.0

##### Accuracy : 0.7113, smoothing : 0.01

##### Accuracy : 0.7034, smoothing : 0.02

##### Accuracy : 0.6978, smoothing : 0.03

##### Accuracy : 0.6934, smoothing : 0.04

##### Accuracy : 0.6897, smoothing : 0.05

## Best_score : 0.7615, Alpha : 0.0

In [621]:
cal_average_of_accuracy(vn_upc_one, vn_upc_more_than_one, [0.9189, 0.6985, 0.7615])

0.7651755834903804


0.7651755834903804

# Upc feature 1 fl

In [622]:
df_to_test = get_df_has_specified_vn_list(vn_upc_one, df=df_train_fl)

In [623]:
test_multinomial(df_to_test)

##### Accuracy : 0.8413, smoothing : 0.0

##### Accuracy : 0.8428, smoothing : 0.01

##### Accuracy : 0.8451, smoothing : 0.02

##### Accuracy : 0.8457, smoothing : 0.03

##### Accuracy : 0.8444, smoothing : 0.04

##### Accuracy : 0.8439, smoothing : 0.05

##### Accuracy : 0.8433, smoothing : 0.06

##### Accuracy : 0.8415, smoothing : 0.07

##### Accuracy : 0.8403, smoothing : 0.08

## Best_score : 0.8457, Alpha : 0.03

In [624]:
X, y = af.get_df_to_fit(df_to_test)
for i in alpha:
    model_nb_dd_one_01 = fitNaiveBayesModel_smoothing(X, y, i)
    y_pred_dd_one = model_nb_dd_one_01.predict(X)
    af.getAccuracy(y, y_pred_dd_one, len(X))
    print(i)

##### Accuracy : 0.832

0.15


##### Accuracy : 0.8235

0.2


##### Accuracy : 0.8179

...

0.8500000000000002


##### Accuracy : 0.7763

0.9000000000000002


##### Accuracy : 0.7735

0.9500000000000003


# Upc feature 2 fl

In [625]:
df_to_test = get_df_has_specified_vn_list(vn_upc_more_than_one, df=df_train_fl)

In [626]:
test_multinomial(df_to_test)

##### Accuracy : 0.7329, smoothing : 0.0

##### Accuracy : 0.7257, smoothing : 0.01

##### Accuracy : 0.7228, smoothing : 0.02

##### Accuracy : 0.7204, smoothing : 0.03

##### Accuracy : 0.7189, smoothing : 0.04

##### Accuracy : 0.7177, smoothing : 0.05

## Best_score : 0.7329, Alpha : 0.0

# Upc feature 3 fl

In [627]:
df_to_test = get_df_has_specified_vn_list(vn_upc_3, is_in=False, df=df_train_fl)

In [628]:
test_multinomial(df_to_test)

##### Accuracy : 0.7731, smoothing : 0.0

##### Accuracy : 0.7463, smoothing : 0.01

##### Accuracy : 0.7396, smoothing : 0.02

##### Accuracy : 0.7344, smoothing : 0.03

##### Accuracy : 0.7299, smoothing : 0.04

##### Accuracy : 0.7262, smoothing : 0.05

## Best_score : 0.7731, Alpha : 0.0

In [629]:
cal_average_of_accuracy(vn_upc_one, vn_upc_more_than_one, [0.8457, 0.7329, 0.7731])

0.7700662952761613


0.7700662952761613

In [630]:
cal_average_of_accuracy(vn_upc_one, vn_upc_more_than_one, [0.9189, 0.7329, 0.7731])

0.7842114937767929


0.7842114937767929

# Upc feature 1 dd+company

In [632]:
df_to_test = get_df_has_specified_vn_list(vn_upc_one, df=df_train_dd)

In [633]:
test_multinomial(df_to_test)

##### Accuracy : 0.8365, smoothing : 0.0

##### Accuracy : 0.8472, smoothing : 0.01

##### Accuracy : 0.8568, smoothing : 0.02

##### Accuracy : 0.8628, smoothing : 0.03

##### Accuracy : 0.869, smoothing : 0.04

...

##### Accuracy : 0.8856, smoothing : 0.12

##### Accuracy : 0.8855, smoothing : 0.13

##### Accuracy : 0.8854, smoothing : 0.14

##### Accuracy : 0.8848, smoothing : 0.15

## Best_score : 0.8856, Alpha : 0.12

# Upc feature 2 dd+company

In [634]:
df_to_test = get_df_has_specified_vn_list(vn_upc_more_than_one, df=df_train_dd)

In [635]:
test_multinomial(df_to_test)

##### Accuracy : 0.645, smoothing : 0.0

##### Accuracy : 0.6162, smoothing : 0.01

##### Accuracy : 0.6101, smoothing : 0.02

##### Accuracy : 0.6056, smoothing : 0.03

##### Accuracy : 0.6026, smoothing : 0.04

##### Accuracy : 0.5994, smoothing : 0.05

## Best_score : 0.645, Alpha : 0.0

# Upc feature 3 dd+company

In [636]:
df_to_test = get_df_has_specified_vn_list(vn_upc_3, is_in=False, df=df_train_dd)

In [637]:
test_multinomial(df_to_test)

##### Accuracy : 0.7378, smoothing : 0.0

##### Accuracy : 0.681, smoothing : 0.01

##### Accuracy : 0.6726, smoothing : 0.02

##### Accuracy : 0.6672, smoothing : 0.03

##### Accuracy : 0.6635, smoothing : 0.04

##### Accuracy : 0.6598, smoothing : 0.05

## Best_score : 0.7378, Alpha : 0.0

In [638]:
cal_average_of_accuracy(vn_upc_one, vn_upc_more_than_one, [0.8856, 0.645, 0.7378])

0.7269718143599727


0.7269718143599727

In [641]:
dd_cols = [col for col in df_dd.columns if col not in vn_and_weekdays_cols]

In [643]:
cp_cols = [col for col in df_train_dd.columns if col not in dd_cols]

In [646]:
len(df_train_dd.columns) - len(cp_cols)

69

In [647]:
df_train_cp = df_train_dd[cp_cols]

# Upc feature 1 company

In [648]:
df_to_test = get_df_has_specified_vn_list(vn_upc_one, df=df_train_cp)

In [649]:
test_multinomial(df_to_test)

##### Accuracy : 0.8283, smoothing : 0.0

##### Accuracy : 0.8312, smoothing : 0.01

##### Accuracy : 0.8309, smoothing : 0.02

##### Accuracy : 0.8301, smoothing : 0.03

##### Accuracy : 0.8287, smoothing : 0.04

##### Accuracy : 0.8267, smoothing : 0.05

##### Accuracy : 0.8245, smoothing : 0.06

## Best_score : 0.8312, Alpha : 0.01

# Upc feature 2 company

In [650]:
df_to_test = get_df_has_specified_vn_list(vn_upc_more_than_one, df=df_train_cp)

In [651]:
test_multinomial(df_to_test)

##### Accuracy : 0.6879, smoothing : 0.0

##### Accuracy : 0.6793, smoothing : 0.01

##### Accuracy : 0.6764, smoothing : 0.02

##### Accuracy : 0.6732, smoothing : 0.03

##### Accuracy : 0.6715, smoothing : 0.04

##### Accuracy : 0.6697, smoothing : 0.05

## Best_score : 0.6879, Alpha : 0.0

# Upc feature 3 company

In [653]:
df_to_test = get_df_has_specified_vn_list(vn_upc_3, is_in=False, df=df_train_cp)

In [654]:
test_multinomial(df_to_test)

##### Accuracy : 0.7459, smoothing : 0.0

##### Accuracy : 0.7139, smoothing : 0.01

##### Accuracy : 0.7067, smoothing : 0.02

##### Accuracy : 0.7009, smoothing : 0.03

##### Accuracy : 0.6973, smoothing : 0.04

##### Accuracy : 0.6938, smoothing : 0.05

## Best_score : 0.7459, Alpha : 0.0

In [655]:
cal_average_of_accuracy(vn_upc_one, vn_upc_more_than_one, [0.8312, 0.6879, 0.7459])

0.7377652080270051


0.7377652080270051

# 현 최고모델 Classification report를 통해 Insight를 얻자

In [35]:
# X_1 = pd.read_csv("Feature_matrix/X_1_201808071357.csv")
# y_1 = pd.read_csv("Feature_matrix/y_1_201808071359.csv")
# X_2 = pd.read_csv("Feature_matrix/X_2_201808071359.csv")
# y_2 = pd.read_csv("Feature_matrix/y_2_201808071402.csv")
# X_3 = pd.read_csv("Feature_matrix/X_3_201808071402.csv")
# y_3 = pd.read_csv("Feature_matrix/y_3_201808071407.csv")

In [55]:
df_to_test = get_df_has_specified_vn_list(vn_test_fl_one, df_dd = df_test_dd, df_fl = df_test_fl, is_need_to_make_df = True)

Concat 완료.


In [56]:
X_1_test = af.get_df_to_fit(df_to_test, is_test_df=True)

In [57]:
X_1_test.drop("index", axis=1, inplace = True)

In [197]:
vn_test_fl_1_2 = vn_test_fl_one + vn_test_fl_more_than_one

In [224]:
df_to_test = get_df_has_specified_vn_list(vn_test_fl_1_2, df_dd=df_test_dd, df_fl=df_test_fl, is_need_to_make_df=True)

Concat 완료.


In [225]:
X_1_2_test = af.get_df_to_fit(df_to_test, True)

In [226]:
X_1_2_test.drop("index", axis=1, inplace = True)

In [168]:
vn_test_fl_2_3 = vn_test_fl_more_than_one + vn_test_fl_dice

In [170]:
df_to_test = get_df_has_specified_vn_list(vn_test_fl_2_3, df = df_test_fl)

In [171]:
X_2_3_test = af.get_df_to_fit(df_to_test, True)

In [172]:
X_2_3_test.drop("index", axis=1, inplace = True)

In [119]:
df_to_test = get_df_has_specified_vn_list(vn_test_fl_more_than_one, df_dd = df_test_dd, df_fl = df_test_fl, is_need_to_make_df = True)

Concat 완료.


In [120]:
X_2_test = af.get_df_to_fit(df_to_test, is_test_df=True)

In [103]:
X_2_test.columns == X_2.columns

array([ True,  True,  True, ...,  True,  True,  True])

In [121]:
X_2_test.drop("index", axis=1, inplace = True)

In [65]:
vn_test_fl_3 = vn_test_fl_one + vn_test_fl_more_than_one

In [122]:
df_to_test = get_df_has_specified_vn_list(vn_test_fl_3, is_in=False, df_dd = df_test_dd, df_fl = df_test_fl, is_need_to_make_df = True)

Concat 완료.


In [123]:
X_3_test = af.get_df_to_fit(df_to_test, is_test_df=True)

In [124]:
X_3_test.drop("index", axis=1, inplace = True)

In [125]:
len(X_1_test), len(X_2_test), len(X_3_test)

(19313, 31216, 45145)

In [126]:
len(X_1_test) + len(X_2_test) + len(X_3_test)

95674

## feature 1 

In [29]:
df_to_test = get_df_has_specified_vn_list(vn_fl_one, df_dd = df_train_dd, df_fl = df_train_fl, is_need_to_make_df = True)

Concat 완료.


In [30]:
X_1, y_1 = af.get_df_to_fit(df_to_test)

In [208]:
model_feature_1 = fitNaiveBayesModel_smoothing(X_1, y_1, 0.14)

In [209]:
y_pred_1 = model_feature_1.predict(X_1)

In [210]:
y_pred_proba_1 = model_feature_1.predict_proba(X_1)

In [211]:
log_loss(y_1, y_pred_proba_1)

0.27655662462465813

In [212]:
af.getAccuracy(y_1, y_pred_1, len(X_1))

##### Accuracy : 0.9212

In [36]:
print(classification_report(y_1, y_pred_1))

             precision    recall  f1-score   support

          3       0.84      1.00      0.91       855
          4       0.97      1.00      0.98       151
          5       0.98      1.00      0.99      2225
          6       0.99      0.95      0.97       565
          7       0.95      0.98      0.97      1253
          8       0.96      0.96      0.96      5560
          9       0.92      0.93      0.92      4969
         12       0.00      0.00      0.00         7
         15       0.55      0.54      0.54        69
         18       0.79      0.63      0.70        54
         19       0.91      0.84      0.87       135
         20       0.81      0.46      0.59        48
         21       0.82      0.72      0.77        39
         22       0.75      0.91      0.82       287
         23       0.88      0.83      0.85        69
         24       0.69      0.92      0.79       270
         25       0.89      0.86      0.88        73
         26       0.77      0.58      0.66   

# feature 1 + 2

In [202]:
vn_fl_1_2 = vn_fl_one + vn_fl_more_than_one

In [203]:
df_to_test = get_df_has_specified_vn_list(vn_fl_1_2, df_dd = df_train_dd, df_fl = df_train_fl, is_need_to_make_df = True)

Concat 완료.


In [204]:
test_multinomial(df_to_test)

##### Accuracy : 0.7498, smoothing : 0.0

##### Accuracy : 0.7377, smoothing : 0.01

##### Accuracy : 0.7349, smoothing : 0.02

##### Accuracy : 0.7337, smoothing : 0.03

##### Accuracy : 0.7325, smoothing : 0.04

##### Accuracy : 0.7314, smoothing : 0.05

## Best_score : 0.7498, Alpha : 0.0

In [205]:
X_1_2, y_1_2 = af.get_df_to_fit(df_to_test)

In [207]:
model_feature_1_2 = fitNaiveBayesModel_smoothing(X_1_2, y_1_2, 0.0)

# feature 2 + 3

In [159]:
vn_fl_2_3 = vn_fl_more_than_one + vn_fl_dice

In [160]:
len(vn_fl_2_3)

76143

In [161]:
df_to_test = get_df_has_specified_vn_list(vn_fl_2_3, df = df_train_fl)

In [162]:
X_2_3, y_2_3 = af.get_df_to_fit(df_to_test)

In [163]:
test_multinomial(df_to_test)

##### Accuracy : 0.6882, smoothing : 0.0

##### Accuracy : 0.6748, smoothing : 0.01

##### Accuracy : 0.6711, smoothing : 0.02

##### Accuracy : 0.6685, smoothing : 0.03

##### Accuracy : 0.6663, smoothing : 0.04

##### Accuracy : 0.6642, smoothing : 0.05

## Best_score : 0.6882, Alpha : 0.0

In [164]:
model_feature_2_3 = fitNaiveBayesModel_smoothing(X_2_3, y_2_3, 0.0)

# feature 2

In [107]:
df_to_test = get_df_has_specified_vn_list(vn_fl_more_than_one, df_dd = df_train_dd, df_fl = df_train_fl, is_need_to_make_df = True)

Concat 완료.


In [37]:
# df_to_test = get_df_has_specified_vn_list(vn_fl_more_than_one, df = df_train_fl)

In [108]:
X_2, y_2 = af.get_df_to_fit(df_to_test)

In [109]:
model_feature_2 = fitNaiveBayesModel_smoothing(X_2, y_2, 0.0)

In [110]:
y_pred_2 = model_feature_2.predict(X_2)

In [111]:
af.getAccuracy(y_2, y_pred_2, len(X_2))

##### Accuracy : 0.7325

In [112]:
print(classification_report(y_2, y_pred_2))

             precision    recall  f1-score   support

          3       0.93      0.94      0.93      2746
          4       0.84      0.97      0.90       123
          5       0.81      0.88      0.85      1336
          6       0.88      0.94      0.91       440
          7       0.80      0.85      0.82      2385
          8       0.74      0.45      0.56      5629
          9       0.81      0.49      0.61      3839
         12       0.71      0.89      0.79        90
         15       0.54      0.86      0.66       208
         18       0.71      0.95      0.82       198
         19       0.81      0.99      0.89       161
         20       0.66      0.93      0.77       312
         21       0.64      0.95      0.77       163
         22       0.71      0.94      0.81       343
         23       0.92      1.00      0.96        58
         24       0.73      0.91      0.81       817
         25       0.67      0.90      0.77      1351
         26       0.76      0.96      0.85   

# feature 3

In [44]:
vn_fl_3 = vn_fl_one + vn_fl_more_than_one

In [113]:
df_to_test = get_df_has_specified_vn_list(vn_fl_3, is_in=False, df_dd = df_train_dd, df_fl = df_train_fl, is_need_to_make_df = True)

Concat 완료.


In [45]:
# df_to_test = get_df_has_specified_vn_list(vn_fl_3, df = df_train_fl, is_in=False)

In [114]:
X_3, y_3 = af.get_df_to_fit(df_to_test)

In [115]:
model_feature_3 = fitNaiveBayesModel_smoothing(X_3, y_3, 0.0)

In [116]:
y_pred_3 = model_feature_3.predict(X_3)

In [117]:
af.getAccuracy(y_3, y_pred_3, len(X_3))

##### Accuracy : 0.7369

In [118]:
print(classification_report(y_3, y_pred_3))

             precision    recall  f1-score   support

          3       0.93      0.88      0.90        42
          4       0.83      0.97      0.90        72
          5       0.82      0.81      0.82      1032
          6       0.87      0.89      0.88       272
          7       0.65      0.71      0.68      2114
          8       0.58      0.31      0.40       972
          9       0.68      0.33      0.45       656
         12       0.69      0.75      0.72       172
         14       1.00      1.00      1.00         4
         15       0.76      0.78      0.77       701
         18       0.77      0.90      0.83       297
         19       0.91      0.95      0.93        79
         20       0.85      0.94      0.89       277
         21       0.82      0.94      0.88       439
         22       0.89      0.89      0.89       298
         23       0.80      1.00      0.89        12
         24       0.77      0.82      0.79      1522
         25       0.83      0.88      0.85   

In [52]:
vn_fl_dice = list(df_to_test["VisitNumber"].unique())

In [53]:
len(vn_fl_dice)

44879

In [174]:
y_pred_test_proba_1 = model_feature_1.predict_proba(X_1_test)
y_pred_test_proba_2_3 = model_feature_2_3.predict_proba(X_2_3_test)

In [227]:
y_pred_test_proba_1_2 = model_feature_1_2.predict_proba(X_1_2_test)

In [127]:
y_pred_test_proba_1 = model_feature_1.predict_proba(X_1_test)
y_pred_test_proba_2 = model_feature_2.predict_proba(X_2_test)
y_pred_test_proba_3 = model_feature_3.predict_proba(X_3_test)

In [303]:
y_pred_proba_1 = model_feature_1.predict_proba(X_1)
y_pred_proba_2 = model_feature_2.predict_proba(X_2)
y_pred_proba_3 = model_feature_3.predict_proba(X_3)

In [169]:
sub_test.set_index("VisitNumber", inplace=True)

In [130]:
def get_proba_and_set_submission(models, features, targets, vns, submission):
    sub = submission.copy()
    sub.set_index("VisitNumber", inplace=True)
    for idx, model in enumerate(models):
        y_pred = model.predict(features[idx])
        y_proba = model.predict_proba(features[idx])
        tt_li = []
        vn_li = vns[idx]
        y_pred_unique = targets[idx].unique()
        for i in range(len(y_pred_unique)):
            tmp = "TripType_" + str(sorted(y_pred_unique)[i])
            tt_li.append(tmp)
        sub.at[vn_li, tt_li] = y_proba
        print(str(idx + 1) + " feature 완료.")
    return sub

In [263]:
sub_test.set_index("VisitNumber", inplace=True)

In [228]:
vn_test_fl_dice = list(df_to_test.VisitNumber.unique())

In [229]:
vns_test = [vn_test_fl_one, vn_test_fl_more_than_one, vn_test_fl_dice]

In [257]:
sub_test = get_proba_and_set_submission(models, features, targets, vns, sub_test)

1 feature 완료.
2 feature 완료.
3 feature 완료.


In [215]:
tt_li = []
for i in range(len(y_1_2.unique())):
    tmp = "TripType_" + str(sorted(y_1_2.unique())[i])
    tt_li.append(tmp)
len(tt_li)

37

In [175]:
submit = submission.copy()

In [177]:
submit_1_test = pd.DataFrame(y_pred_test_proba_1, columns = tt_li)

In [178]:
submit_1_test["VisitNumber"] = vn_test_fl_one

In [179]:
submit_1_test["TripType_14"] = 0
submit_1_test["TripType_43"] = 0
submit_1_test["TripType_44"] = 0

In [180]:
submit_1_test = submit_1_test[submit.columns]

In [228]:
submit_1_2_test = pd.DataFrame(y_pred_test_proba_1_2, columns = tt_li)

In [229]:
submit_1_2_test["TripType_14"] = 0

In [230]:
len(vn_test_fl_1_2), len(submit_1_2_test)

(50529, 50529)

In [231]:
submit_1_2_test["VisitNumber"] = vn_test_fl_1_2

In [232]:
submit_1_2_test = submit_1_2_test[submit.columns]

In [182]:
submit_2_3_test = pd.DataFrame(y_pred_test_proba_2_3, columns = tt_li)

In [183]:
submit_2_3_test["VisitNumber"] = vn_test_fl_2_3

In [184]:
submit_2_3_test = submit_2_3_test[submit.columns]

In [185]:
submit_test_all_2_3 = pd.concat([submit_1_test, submit_2_3_test])

In [186]:
submit_test_all_2_3 = submit_test_all_2_3.sort_values("VisitNumber")
submit_test_all_2_3.reset_index(drop=True, inplace=True)

In [189]:
af.saveDataFrameToCsv(submit_test_all_2_3, "last_try")

In [233]:
submit_test_all_1_2 = pd.concat([submit_1_2_test, submit_3_test])

In [234]:
submit_test_all_1_2 = submit_test_all_1_2.sort_values("VisitNumber")
submit_test_all_1_2.reset_index(drop=True, inplace=True)

In [236]:
af.saveDataFrameToCsv(submit_test_all_1_2, "get_insight")

In [190]:
df_to_test = get_concat_df_for_test_df(df_train_dd, df_train_fl)

In [192]:
X_1.head()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,1627.0,-1,8191.0,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [193]:
X_test, y_test = af.get_df_to_fit(df_to_test)

In [194]:
X_test.head()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,1627.0,-1,8191.0,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [195]:
y_pred = model_feature_1.predict(X_test)

In [196]:
af.getAccuracy(y_test, y_pred, len(X_test))

##### Accuracy : 0.4634

In [136]:
submit_2_test = pd.DataFrame(y_pred_test_proba_2, columns = tt_li)

In [137]:
submit_2_test["VisitNumber"] = vn_test_fl_more_than_one

In [138]:
submit_2_test["TripType_14"] = 0

In [139]:
submit_2_test = submit_2_test[submit.columns]

In [140]:
vn_test_fl_dice = list(df_to_test.VisitNumber.unique())

In [141]:
len(vn_test_fl_dice)

45145

In [143]:
submit_3_test = pd.DataFrame(y_pred_test_proba_3, columns = tt_li)

In [144]:
submit_3_test["VisitNumber"] = vn_test_fl_dice

In [145]:
submit_3_test = submit_3_test[submit.columns]

In [147]:
submit_test_all = pd.concat([submit_1_test, submit_2_test, submit_3_test])

In [148]:
submit_test_all = submit_test_all.sort_values("VisitNumber")
submit_test_all.reset_index(drop=True, inplace=True)

In [152]:
submit_test_all.head()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,1.106492e-50,9.653557e-54,5.636323e-16,9.078964e-24,2.812958e-04,1.979547e-02,1.340561e-15,6.397182e-43,0.000000e+00,...,1.127518e-16,2.911286e-13,0.968618,0.011305,4.639459e-13,7.853481e-35,2.124187e-25,6.522341e-25,2.901157e-23,2.432760e-23
1,2,3.650732e-48,9.982242e-07,1.871108e-11,2.094947e-29,1.702872e-31,3.806059e-28,2.689917e-18,4.188307e-30,8.873491e-57,...,2.052912e-22,2.514577e-03,0.473402,0.143742,8.879635e-03,1.624739e-19,2.273621e-11,3.045582e-08,3.913261e-13,1.550635e-31
2,3,1.210754e-03,1.512679e-03,1.532167e-02,5.597487e-03,2.832801e-02,3.347708e-02,2.256775e-02,1.899446e-03,3.817823e-05,...,2.203266e-02,1.146737e-02,0.018215,0.070369,1.989943e-02,3.978709e-03,1.161236e-02,7.972996e-03,6.121550e-03,5.751394e-01
3,4,1.785875e-04,7.326824e-05,3.609887e-05,1.085832e-04,6.762326e-05,2.699233e-05,7.892577e-01,7.691106e-08,0.000000e+00,...,1.774390e-05,6.892610e-06,0.000006,0.000012,2.456320e-07,9.968076e-08,4.435841e-06,0.000000e+00,0.000000e+00,1.955480e-03
4,6,1.210754e-03,1.512679e-03,1.532167e-02,5.597487e-03,2.832801e-02,3.347708e-02,2.256775e-02,1.899446e-03,3.817823e-05,...,2.203266e-02,1.146737e-02,0.018215,0.070369,1.989943e-02,3.978709e-03,1.161236e-02,7.972996e-03,6.121550e-03,5.751394e-01


In [153]:
submit_to_check.head()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,3.175828e-44,1.002715e-40,2.166095e-13,4.179372e-22,6.724353e-03,1.363274e-01,2.948011e-13,1.035182e-41,0.000000e+00,...,1.666031e-13,2.328031e-12,0.826118,0.030830,1.237613e-12,7.923613e-33,1.815581e-23,3.778327e-23,2.590057e-22,5.927163e-22
1,2,9.340914e-37,6.497091e-08,1.836845e-11,4.432472e-30,5.931267e-32,8.130657e-30,1.061700e-18,1.666864e-30,1.255418e-36,...,2.480656e-22,3.706382e-03,0.202063,0.026589,9.295111e-03,6.476055e-20,2.154178e-11,5.776768e-09,3.698196e-13,2.854383e-31
2,3,1.477099e-03,1.905515e-03,1.947489e-02,7.086649e-03,3.607914e-02,3.982477e-02,2.684251e-02,2.461405e-03,5.035697e-05,...,2.841871e-02,1.510282e-02,0.023757,0.091342,2.647738e-02,5.176118e-03,1.514823e-02,1.023443e-02,8.074890e-03,4.568377e-01
3,4,1.785875e-04,7.326824e-05,3.609887e-05,1.085832e-04,6.762326e-05,2.699233e-05,7.892577e-01,7.691106e-08,0.000000e+00,...,1.774390e-05,6.892610e-06,0.000006,0.000012,2.456320e-07,9.968076e-08,4.435841e-06,0.000000e+00,0.000000e+00,1.955480e-03
4,6,1.477099e-03,1.905515e-03,1.947489e-02,7.086649e-03,3.607914e-02,3.982477e-02,2.684251e-02,2.461405e-03,5.035697e-05,...,2.841871e-02,1.510282e-02,0.023757,0.091342,2.647738e-02,5.176118e-03,1.514823e-02,1.023443e-02,8.074890e-03,4.568377e-01


In [151]:
af.saveDataFrameToCsv(submit_test_all, "submit_test_all_2")

In [97]:
submit_to_check = pd.read_csv("Feature_matrix/submit_please_get_great_score_201808080131.csv")

In [434]:
X, y = af.get_df_to_fit(df_train_fl)

In [436]:
model_test = fitNaiveBayesModel_smoothing(X, y, 0.0)

In [437]:
y_pred = model_test.predict(X)

In [438]:
af.getAccuracy(y, y_pred, len(X))

##### Accuracy : 0.6993

In [439]:
log_loss(y, model_test.predict_proba(X))

1.0689111273514562

In [429]:
af.saveDataFrameToCsv(submit_test_all, "submit_please_get_great_score")

In [337]:
submit_1 = pd.DataFrame(y_pred_proba_1, columns = tt_li)

In [338]:
submit_1["VisitNumber"] = vn_fl_one

In [339]:
submit_1["TripType_14"] = 0

In [340]:
submit_1["TripType_43"] = 0

In [341]:
submit_1["TripType_44"] = 0

In [331]:
sub_test.drop("TripType", axis =1 , inplace=True)

In [333]:
sub_test=sub_test.reset_index()

In [344]:
submit_1 = submit_1[sub_test.columns]

In [347]:
submit_2 = pd.DataFrame(y_pred_proba_2, columns = tt_li)

In [348]:
submit_2["TripType_14"] = 0

In [349]:
submit_2["VisitNumber"] = vn_fl_more_than_one

In [352]:
submit_2 = submit_2[sub_test.columns]

In [354]:
submit_3 = pd.DataFrame(y_pred_proba_3, columns = tt_li)

In [364]:
vn_fl_dice = list(df_to_test.VisitNumber.unique())

In [365]:
submit_3["VisitNumber"] = vn_fl_dice

In [367]:
submit_3 = submit_3[sub_test.columns]

In [371]:
submit_all = pd.concat([submit_1, submit_2, submit_3])

In [373]:
submit_all = submit_all.sort_values("VisitNumber")

In [377]:
submit_all.reset_index(drop=True, inplace=True)

In [379]:
submit_all.set_index("VisitNumber", inplace=True)

In [279]:
sub_test[sub_test.index.isin(vn_fl_one)].values[0]

array([1.41254359e-06, 5.79517666e-07, 2.85525253e-07, 6.99342921e-06,
       5.34868521e-07, 9.51288050e-01, 2.52301732e-07, 6.08330653e-10,
       7.52377882e-10, 9.56470393e-08, 1.09051498e-07, 5.89798632e-07,
       1.14668745e-07, 9.76530584e-08, 6.54228115e-07, 3.42084355e-07,
       3.67567079e-07, 1.02254938e-07, 1.43131129e-07, 1.08304044e-07,
       1.80553735e-07, 6.11786011e-08, 5.39045447e-07, 5.89872472e-07,
       7.51374501e-08, 5.87045639e-08, 6.14542455e-08, 4.84732605e-02,
       1.40345978e-07, 5.45173372e-08, 5.03687637e-08, 9.36357698e-08,
       1.94283506e-09, 7.88428341e-10, 3.50854360e-08, 5.71881452e-09,
       7.52377882e-10, 2.23956759e-04])

In [446]:
submit_test_all.set_index("VisitNumber", inplace=True)

In [230]:
submit_fl = get_proba_and_set_submission(models, features_test, targets, vns_test, submission)

1 feature 완료.
2 feature 완료.
3 feature 완료.


In [234]:
submit_fl = submit_fl.reset_index()

In [236]:
af.saveDataFrameToCsv(submit_fl, "submit_nb_tried_three_features")

In [574]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder().fit(y_1)
y_labeled = label_enc.transform(y_1)

In [575]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_labeled, random_state=0)

In [576]:
dtrain = xgb.DMatrix(X_train.values, label=y_train)
dtest = xgb.DMatrix(X_test.values, label=y_test)

In [578]:
num_boost_round = 300
params = {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss',
          'num_class': 35, 
          'max_delta_step': 3, 
          'eta': 0.2}

evals = [(dtrain, 'train'), (dtest, 'eval')]

bst = xgb.train(params=params,  
                dtrain=dtrain, 
                num_boost_round=num_boost_round, 
                evals=evals,
                early_stopping_rounds=10,)

[0]	train-mlogloss:2.9518	eval-mlogloss:2.95404
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:2.41875	eval-mlogloss:2.4239
[2]	train-mlogloss:1.98923	eval-mlogloss:1.99831
[3]	train-mlogloss:1.66572	eval-mlogloss:1.67476
[4]	train-mlogloss:1.44663	eval-mlogloss:1.45736
[5]	train-mlogloss:1.28503	eval-mlogloss:1.30006
[6]	train-mlogloss:1.15639	eval-mlogloss:1.1728
[7]	train-mlogloss:1.05341	eval-mlogloss:1.07154
[8]	train-mlogloss:0.969075	eval-mlogloss:0.989787
[9]	train-mlogloss:0.898572	eval-mlogloss:0.921121
[10]	train-mlogloss:0.837392	eval-mlogloss:0.860966
[11]	train-mlogloss:0.786237	eval-mlogloss:0.811636
[12]	train-mlogloss:0.742256	eval-mlogloss:0.770075
[13]	train-mlogloss:0.70336	eval-mlogloss:0.731789
[14]	train-mlogloss:0.669246	eval-mlogloss:0.698751
[15]	train-mlogloss:0.640082	eval-mlogloss:0.670561
[16]	train-mlogloss:0.614212	eval-mlogloss:0.64

[155]	train-mlogloss:0.266862	eval-mlogloss:0.367576
[156]	train-mlogloss:0.266475	eval-mlogloss:0.367413
[157]	train-mlogloss:0.266077	eval-mlogloss:0.367257
[158]	train-mlogloss:0.265656	eval-mlogloss:0.367059
[159]	train-mlogloss:0.265275	eval-mlogloss:0.366842
[160]	train-mlogloss:0.26489	eval-mlogloss:0.366648
[161]	train-mlogloss:0.26444	eval-mlogloss:0.366539
[162]	train-mlogloss:0.264052	eval-mlogloss:0.366425
[163]	train-mlogloss:0.263638	eval-mlogloss:0.366241
[164]	train-mlogloss:0.263185	eval-mlogloss:0.366225
[165]	train-mlogloss:0.262759	eval-mlogloss:0.366192
[166]	train-mlogloss:0.262383	eval-mlogloss:0.366134
[167]	train-mlogloss:0.262007	eval-mlogloss:0.365964
[168]	train-mlogloss:0.261601	eval-mlogloss:0.365746
[169]	train-mlogloss:0.26112	eval-mlogloss:0.365508
[170]	train-mlogloss:0.260762	eval-mlogloss:0.365492
[171]	train-mlogloss:0.260425	eval-mlogloss:0.365353
[172]	train-mlogloss:0.260108	eval-mlogloss:0.365299
[173]	train-mlogloss:0.259741	eval-mlogloss:0.365

In [600]:
af.saveDataFrameToCsv(X_1, "X_1")
af.saveDataFrameToCsv(y_1, "y_1")
af.saveDataFrameToCsv(X_2, "X_2")
af.saveDataFrameToCsv(y_2, "y_2")
af.saveDataFrameToCsv(X_3, "X_3")
af.saveDataFrameToCsv(y_3, "y_3")

In [579]:
label_enc_2 = LabelEncoder().fit(y_2)
y_labeled = label_enc_2.transform(y_2)

In [580]:
X_train, X_test, y_train, y_test = train_test_split(X_2, y_labeled, random_state=0)

In [581]:
dtrain = xgb.DMatrix(X_train.values, label=y_train)
dtest = xgb.DMatrix(X_test.values, label=y_test)

In [ ]:
num_boost_round = 50
params = {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss',
          'num_class': 37, 
          'max_delta_step': 3, 
          'eta': 0.2}

evals = [(dtrain, 'train'), (dtest, 'eval')]

bst_2 = xgb.train(params=params,  
                dtrain=dtrain, 
                num_boost_round=num_boost_round, 
                evals=evals,
                early_stopping_rounds=10,)

In [ ]:
label_enc_3 = LabelEncoder().fit(y_3)
y_labeled = label_enc_3.transform(y_3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_3, y_labeled, random_state=0)

In [ ]:
dtrain = xgb.DMatrix(X_train.values, label=y_train)
dtest = xgb.DMatrix(X_test.values, label=y_test)

In [ ]:
num_boost_round = 50
params = {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss',
          'num_class': 38, 
          'max_delta_step': 3, 
          'eta': 0.2}

evals = [(dtrain, 'train'), (dtest, 'eval')]

bst_3 = xgb.train(params=params,  
                dtrain=dtrain, 
                num_boost_round=num_boost_round, 
                evals=evals,
                early_stopping_rounds=10,)